In [1]:
# Bloco 1: Preparar os Dados
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
import os
data = pd.read_csv('D:\\dados\\bar_M15_V03_data_01-01-2023_a_31-08-2024.csv')
data['DateTime'] = pd.to_datetime(data['DateTime'])
# filtra o dataframe para pegar apenas o mês de 08 de 2024
#data = data[(data['DateTime'] >= '2024-08-01') & (data['DateTime'] <= '2024-08-31')]

# Criar a coluna "Valor", que é uma cópia de "Close" e não será normalizada
data['Valor'] = data['Close']

# Normalizar as colunas necessárias (exceto "Valor" e "Gatilho")
scaler = MinMaxScaler()
cols_to_normalize = [
    'Open', 'High', 'Low', 'Close', 'Volume', 'PavioSuperior', 'PavioInferior',
    'Corpo', 'Range','SMA4','SMA8','SMA12','SMA20', 'SMA50', 'SMA100', 'SMA200', 'StochasticoK',
    'StochasticoD', 'RSI', 'MACD', 'MACDSignal', 'MACDHistogram','atr8','atr14','atr28','dayO','dayH','dayL'
]
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])

# Converter todos os valores para tipo float32 para evitar problemas de tipo
data = data.astype({col: 'float32' for col in cols_to_normalize + ['Valor']})

import gym
from gym import spaces

class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data.reset_index(drop=True)
        self.current_step = 0
        self.position = 0  # 0 = neutro, 1 = comprado, -1 = vendido
        self.entry_price = 0.0
        self.entry_step = None
        self.entry_datetime = None  # Novo atributo para armazenar o DateTime de entrada
        self.action_space = spaces.Discrete(3)  # 0 = Manter, 1 = Comprar, 2 = Vender
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(len(data.columns) - 3 + 1,), dtype=np.float32
        )
        self.trades = []  # Lista para armazenar as operações realizadas

        # Alteração 1: Variáveis para Recompensa e Lucro Reais
        self.reward_real = 0.0
        self.profit_real = 0.0

    def reset(self):
        self.current_step = 0
        self.position = 0
        self.entry_price = 0.0
        self.entry_step = None
        self.entry_datetime = None
        self.trades = []
        self.reward_real = 0.0  # Resetar recompensa real
        self.profit_real = 0.0  # Resetar lucro real
        return self._next_observation()

    def _next_observation(self):
        obs = self.data.iloc[self.current_step].drop(['Valor', 'DateTime', 'Gatilho']).values
        obs = np.append(obs, self.position)  # Incluir a posição atual na observação
        return obs.astype(np.float32)

    def step(self, action):
        done = self.current_step >= len(self.data) - 2  # Ajustado para evitar índice fora do intervalo
        info = {}

        # Obter o valor atual e o próximo valor
        current_price = self.data['Valor'].iloc[self.current_step]
        next_price = self.data['Valor'].iloc[self.current_step + 1]
        price_change = next_price - current_price

        # Obter o valor do gatilho no passo atual
        gatilho = int(self.data['Gatilho'].iloc[self.current_step])

        # Alteração 2: Inicializar variáveis para recompensa e lucro reais
        profit_real = 0  # Lucro real da operação
        reward_real = 0  # Recompensa real da operação
        reward = 0       # Recompensa modificada para o agente
        penalty_factor = 2.0  # Fator de penalidade (pode ajustar conforme necessário)

        # Se o gatilho estiver ativo, o agente pode executar todas as ações
        if gatilho == 1:
            if action == 1:  # Comprar
                if self.position == 0:
                    self.position = 1  # Abrir posição comprada
                    self.entry_price = current_price
                    self.entry_step = self.current_step
                    self.entry_datetime = self.data['DateTime'].iloc[self.current_step]
                    reward -= 0.25  # Custo de operação
                    reward_real -= 0.25  # Custo real
                    info['trade'] = {
                        'type': 'buy',
                        'entry_step': self.entry_step,
                        'entry_price': self.entry_price,
                        'entry_datetime': self.entry_datetime
                    }
                elif self.position == -1:
                    # Fechar posição vendida
                    self.exit_price = current_price
                    profit_real = self.entry_price - self.exit_price - 0.25  # Ganho da posição vendida
                    self.profit_real += profit_real
                    reward_real += profit_real
                    # Aplicar punição mais severa se houve perda
                    if profit_real < 0:
                        reward += profit_real * penalty_factor
                    else:
                        reward += profit_real
                    self.exit_step = self.current_step
                    self.exit_datetime = self.data['DateTime'].iloc[self.current_step]
                    info['trade'] = {
                        'type': 'close_short',
                        'exit_step': self.exit_step,
                        'exit_price': self.exit_price,
                        'exit_datetime': self.exit_datetime,
                        'profit': profit_real
                    }
                    # Registrar a operação
                    self.trades.append({
                        'type': 'sell',
                        'entry_step': self.entry_step,
                        'entry_price': self.entry_price,
                        'entry_datetime': self.entry_datetime,
                        'exit_step': self.exit_step,
                        'exit_price': self.exit_price,
                        'exit_datetime': self.exit_datetime,
                        'profit': profit_real,
                        'reward_real': reward_real
                    })
                    # Resetar posição
                    self.position = 0
                    self.entry_step = None
                    self.entry_datetime = None
            elif action == 2:  # Vender
                if self.position == 0:
                    self.position = -1  # Abrir posição vendida
                    self.entry_price = current_price
                    self.entry_step = self.current_step
                    self.entry_datetime = self.data['DateTime'].iloc[self.current_step]
                    reward -= 0.25  # Custo de operação
                    reward_real -= 0.25  # Custo real
                    info['trade'] = {
                        'type': 'sell',
                        'entry_step': self.entry_step,
                        'entry_price': self.entry_price,
                        'entry_datetime': self.entry_datetime
                    }
                elif self.position == 1:
                    # Fechar posição comprada
                    self.exit_price = current_price
                    profit_real = self.exit_price - self.entry_price - 0.25  # Ganho da posição comprada
                    self.profit_real += profit_real
                    reward_real += profit_real
                    # Aplicar punição mais severa se houve perda
                    if profit_real < 0:
                        reward += profit_real * penalty_factor
                    else:
                        reward += profit_real
                    self.exit_step = self.current_step
                    self.exit_datetime = self.data['DateTime'].iloc[self.current_step]
                    info['trade'] = {
                        'type': 'close_long',
                        'exit_step': self.exit_step,
                        'exit_price': self.exit_price,
                        'exit_datetime': self.exit_datetime,
                        'profit': profit_real
                    }
                    # Registrar a operação
                    self.trades.append({
                        'type': 'buy',
                        'entry_step': self.entry_step,
                        'entry_price': self.entry_price,
                        'entry_datetime': self.entry_datetime,
                        'exit_step': self.exit_step,
                        'exit_price': self.exit_price,
                        'exit_datetime': self.exit_datetime,
                        'profit': profit_real,
                        'reward_real': reward_real
                    })
                    # Resetar posição
                    self.position = 0
                    self.entry_step = None
                    self.entry_datetime = None
            else:  # Manter
                pass  # Nenhuma ação necessária
        else:  # Gatilho == 0, nenhuma posição deve ser mantida
            # Se há uma posição aberta, fechá-la
            if self.position == 1:  # Fechar posição comprada
                self.exit_price = current_price
                profit_real = self.exit_price - self.entry_price - 0.25  # Ganho da posição comprada
                self.profit_real += profit_real
                reward_real += profit_real
                # Aplicar punição mais severa se houve perda
                if profit_real < 0:
                    reward += profit_real * penalty_factor
                else:
                    reward += profit_real
                self.exit_step = self.current_step
                self.exit_datetime = self.data['DateTime'].iloc[self.current_step]
                info['trade'] = {
                    'type': 'close_long',
                    'exit_step': self.exit_step,
                    'exit_price': self.exit_price,
                    'exit_datetime': self.exit_datetime,
                    'profit': profit_real
                }
                # Registrar a operação
                self.trades.append({
                    'type': 'buy',
                    'entry_step': self.entry_step,
                    'entry_price': self.entry_price,
                    'entry_datetime': self.entry_datetime,
                    'exit_step': self.exit_step,
                    'exit_price': self.exit_price,
                    'exit_datetime': self.exit_datetime,
                    'profit': profit_real,
                    'reward_real': reward_real
                })
                # Resetar posição
                self.position = 0
                self.entry_step = None
                self.entry_datetime = None
            elif self.position == -1:  # Fechar posição vendida
                self.exit_price = current_price
                profit_real = self.entry_price - self.exit_price - 0.25  # Ganho da posição vendida
                self.profit_real += profit_real
                reward_real += profit_real
                # Aplicar punição mais severa se houve perda
                if profit_real < 0:
                    reward += profit_real * penalty_factor
                else:
                    reward += profit_real
                self.exit_step = self.current_step
                self.exit_datetime = self.data['DateTime'].iloc[self.current_step]
                info['trade'] = {
                    'type': 'close_short',
                    'exit_step': self.exit_step,
                    'exit_price': self.exit_price,
                    'exit_datetime': self.exit_datetime,
                    'profit': profit_real
                }
                # Registrar a operação
                self.trades.append({
                    'type': 'sell',
                    'entry_step': self.entry_step,
                    'entry_price': self.entry_price,
                    'entry_datetime': self.entry_datetime,
                    'exit_step': self.exit_step,
                    'exit_price': self.exit_price,
                    'exit_datetime': self.exit_datetime,
                    'profit': profit_real,
                    'reward_real': reward_real
                })
                # Resetar posição
                self.position = 0
                self.entry_step = None
                self.entry_datetime = None

        # Atualizar as recompensas reais
        self.reward_real += reward_real

        # Atualizar o passo atual
        self.current_step += 1

        obs = self._next_observation()
        return obs, reward, done, info


# Bloco 3: Criar o Agente DQN usando PyTorch

import torch
import torch.nn as nn
import torch.optim as optim
import collections
import random

# Configurações do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Criar o ambiente
env = TradingEnv(data)

obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

# Definir a rede DQN
class DQN(nn.Module):
    def __init__(self, obs_size, n_actions):
        super(DQN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(obs_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.model(x)

# Instanciar a rede
q_net = DQN(obs_size, n_actions).to(device)
target_net = DQN(obs_size, n_actions).to(device)
target_net.load_state_dict(q_net.state_dict())

# Definir o otimizador
optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

# Hiperparâmetros para DQN
memory_size = 10000
batch_size = 64
gamma = 0.99
epsilon_start = 0.5
epsilon_end = 0.05
epsilon_decay = 0.99
target_update = 10  # Atualizar a rede alvo a cada 10 episódios

# Inicializar a memória de replay
memory = collections.deque(maxlen=memory_size)

# Função para selecionar ação usando epsilon-greedy
def select_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice([0, 1, 2])
    else:
        with torch.no_grad():
            q_values = q_net(state)
            return q_values.argmax().item()


# Bloco 4: Treinamento do Agente DQN com Salvamento dos Melhores Episódios Após Cada Episódio

num_episodes = 100  # Defina o número de episódios de treinamento
epsilon = epsilon_start
best_episodes = []

save_dir = "4.14"
os.makedirs(save_dir, exist_ok=True)

for episode in range(num_episodes):
    start_time = time.time()
    obs = env.reset()
    obs = torch.FloatTensor(obs).unsqueeze(0).to(device)
    done = False
    total_reward = 0
    total_reward_real = 0  # Total da recompensa real
    total_profit_real = 0  # Total do lucro real
    steps = 0
    actions_count = {0: 0, 1: 0, 2: 0}
    wins = 0
    losses = 0
    win_total = 0
    lose_total = 0
    trades = []
    current_trade = None

    while not done:
        steps += 1

        # Selecionar ação
        action = select_action(obs, epsilon)

        # Executar ação no ambiente
        obs_next, reward, done, info = env.step(action)
        obs_next = torch.FloatTensor(obs_next).unsqueeze(0).to(device)

        # Armazenar na memória de replay
        memory.append((obs, action, reward, obs_next, done))

        # Atualizar o estado
        obs = obs_next

        # Atualizar contagem de ações
        actions_count[action] += 1

        # Atualizar recompensa total
        total_reward += reward

        # Atualizar recompensas e lucros reais
        total_reward_real = env.reward_real
        total_profit_real = env.profit_real

        # Processar informações de trade
        if 'trade' in info:
            trade_info = info['trade']
            if trade_info['type'] in ['buy', 'sell']:
                # Início de uma nova operação
                current_trade = {
                    'type': trade_info['type'],
                    'entry_step': trade_info['entry_step'],
                    'entry_price': trade_info['entry_price'],
                    'entry_datetime': trade_info['entry_datetime'],
                    'exit_step': None,
                    'exit_price': None,
                    'exit_datetime': None,
                    'profit': None
                }
            elif trade_info['type'] in ['close_long', 'close_short']:
                # Fechamento de uma operação existente
                current_trade['exit_step'] = trade_info['exit_step']
                current_trade['exit_price'] = trade_info['exit_price']
                current_trade['exit_datetime'] = trade_info['exit_datetime']
                current_trade['profit'] = trade_info['profit']
                trades.append(current_trade.copy())
                # Atualizar ganhos e perdas
                if current_trade['profit'] > 0:
                    wins += 1
                    win_total += current_trade['profit']
                elif current_trade['profit'] < 0:
                    losses += 1
                    lose_total += current_trade['profit']
                current_trade = None

        # Treinar a rede se a memória tiver tamanho suficiente
        if len(memory) >= batch_size:
            batch = random.sample(memory, batch_size)
            states, actions_batch, rewards_batch, next_states, dones = zip(*batch)

            states = torch.cat(states).to(device)
            actions_batch = torch.tensor(actions_batch, dtype=torch.long, device=device).unsqueeze(1)
            rewards_batch = torch.tensor(rewards_batch, dtype=torch.float32, device=device).unsqueeze(1)
            next_states = torch.cat(next_states).to(device)
            dones = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

            # Computar Q-valor atual
            q_values = q_net(states).gather(1, actions_batch)

            # Computar Q-valor alvo usando a rede alvo
            with torch.no_grad():
                next_q_values = target_net(next_states).max(1)[0].unsqueeze(1)
            target_q_values = rewards_batch + gamma * next_q_values * (1 - dones)

            # Calcular a perda
            loss = nn.MSELoss()(q_values, target_q_values)

            # Otimizar a rede
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Decaimento de epsilon
    epsilon = max(epsilon_end, epsilon_decay * epsilon)

    # Atualizar a rede alvo
    if episode % target_update == 0:
        target_net.load_state_dict(q_net.state_dict())

    # Cálculo do tempo de treinamento do episódio
    end_time = time.time()
    episode_time = end_time - start_time

    win_rate = wins / (wins + losses) if (wins + losses) > 0 else 0
    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward:.2f}, "
          f"Total Real Reward: {total_reward_real:.2f}, Total Real Profit: {total_profit_real:.2f}, "
          f"Win Rate: {win_rate:.2f}, Wins: {wins}, Losses: {losses}, Epsilon: {epsilon:.4f}, Steps: {steps}, Time: {episode_time:.2f}s")
    print(f"Ações: Manter={actions_count[0]}, Comprar={actions_count[1]}, Vender={actions_count[2]}")
    print(f"Ganhos Totais: {win_total:.2f}, Perdas Totais: {lose_total:.2f}")

    # Salvar informações do episódio
    episode_info = {
        'episode': episode + 1,
        'total_reward': total_reward,
        'total_reward_real': total_reward_real,
        'total_profit_real': total_profit_real,
        'win_rate': win_rate,
        'wins': wins,
        'losses': losses,
        'actions_count': actions_count.copy(),
        'win_total': win_total,
        'lose_total': lose_total,
        'steps': steps,
        'episode_time': episode_time,
        'model_state_dict': q_net.state_dict(),
        'trades': trades.copy()  # Salvar as operações do episódio
    }

    # Adicionar o episódio à lista e manter os top 10
    best_episodes.append(episode_info)
    best_episodes = sorted(best_episodes, key=lambda x: x['total_reward'], reverse=True)[:10]

    # Salvar o modelo e log se o episódio for um dos top 10
    if episode_info in best_episodes:
        model_path = os.path.join(save_dir, f"model_episode_{episode_info['episode']}.pth")
        torch.save(episode_info['model_state_dict'], model_path)
        episode_info['model_path'] = model_path

        # Salvar o log completo das operações
        log_path = os.path.join(save_dir, f"log_episode_{episode_info['episode']}.csv")
        trades_df = pd.DataFrame(episode_info['trades'])
        trades_df.to_csv(log_path, index=False)
        episode_info['log_path'] = log_path

        print(f"Modelo e log do episódio {episode_info['episode']} salvos em: {model_path} e {log_path}\n")

print("\nTreinamento finalizado.")
print("Top 10 Melhores Episódios:")
for idx, ep in enumerate(best_episodes, 1):
    print(f"Rank {idx}: Episode {ep['episode']}, Total Reward: {ep['total_reward']:.2f}, "
          f"Total Real Reward: {ep['total_reward_real']:.2f}, Total Real Profit: {ep['total_profit_real']:.2f}, "
          f"Win Rate: {ep['win_rate']:.2f}, Wins: {ep['wins']}, Losses: {ep['losses']}, "
          f"Ações: {ep['actions_count']}, Steps: {ep['steps']}, Time: {ep['episode_time']:.2f}s")



Episode 1/100, Total Reward: -33595.25, Total Real Reward: 380.00, Total Real Profit: 992.50, Win Rate: 0.52, Wins: 1267, Losses: 1173, Epsilon: 0.4950, Steps: 36754, Time: 178.97s
Ações: Manter=14326, Comprar=10225, Vender=12203
Ganhos Totais: 34967.75, Perdas Totais: -33975.25
Modelo e log do episódio 1 salvos em: 4.14\model_episode_1.pth e 4.14\log_episode_1.csv

Episode 2/100, Total Reward: -41720.25, Total Real Reward: -3901.75, Total Real Profit: -3324.25, Win Rate: 0.50, Wins: 1158, Losses: 1146, Epsilon: 0.4900, Steps: 36754, Time: 184.50s
Ações: Manter=13168, Comprar=10372, Vender=13214
Ganhos Totais: 34494.25, Perdas Totais: -37818.50
Modelo e log do episódio 2 salvos em: 4.14\model_episode_2.pth e 4.14\log_episode_2.csv

Episode 3/100, Total Reward: -38648.50, Total Real Reward: -2206.00, Total Real Profit: -1634.00, Win Rate: 0.51, Wins: 1151, Losses: 1122, Epsilon: 0.4851, Steps: 36754, Time: 189.96s
Ações: Manter=13336, Comprar=10773, Vender=12645
Ganhos Totais: 34808.50,